# Lab 03 - Introduction to Natural Language Processing 01

## FastText

### Question 1

In [ ]:
from datasets import load_dataset

dataset = load_dataset("imdb")
dataset

In [ ]:
dataset_shuffle = dataset.shuffle(seed=42)

In [ ]:
import fasttext

In [ ]:
from lib.util import lower_and_remove_punctuation

def format_for_fasttext(dataset_key : str):
    """TODO
    """
    with open(dataset_key + ".txt", 'w', encoding="utf-8") as f:
        def write_to_file(element : dict) -> dict:
            """TODO
            """
            f.write("__label__" + str(element['label']) + " " +  lower_and_remove_punctuation(element)['text'] + '\n')
            return element

        dataset_shuffle[dataset_key].map(write_to_file, load_from_cache_file=False)

In [ ]:
format_for_fasttext('train')
format_for_fasttext('test')

### Question 2

In [ ]:
model = fasttext.train_supervised("train.txt")
model

In [ ]:
model.test_label("test.txt")

### Question 3

In [ ]:
ratio = 0.2
num_rows = dataset["train"].num_rows

with open("train.txt") as input:
    lines = input.readlines()

    with open("validation.txt", "w") as validation:
        for l in lines[: int(ratio * num_rows)]:
            validation.write(l)

    with open("train2.txt", "w", encoding="utf-8") as train:
        for l in lines[int(ratio * num_rows): ]:
            train.write(l)

In [ ]:
model_autotuned = fasttext.train_supervised(input='train2.txt', autotuneValidationFile='validation.txt')

In [ ]:
model_autotuned.test_label("test.txt")

### Question 4

In [ ]:
model.test_label("test.txt")

In [ ]:
model_autotuned.test_label("test.txt")